# Acquire Data from BGG's API

I pulled data from all board games on BoardGameGeek.com with around 1000+ user reviews. At the time that I extracted the data, I was still unsure about what kind of analytic approach I would be taking. I wanted to extract only the games with a substantial number of reviews, just in case I ended up having to do something like conduct a separate analysis (e.g., LDA) within each game. 

Much of this code was adapted from: http://sdsawtelle.github.io/blog/output/boardgamegeek-data-scraping.html

In [1]:
import requests
from bs4 import BeautifulSoup
import scipy.io
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import numpy as np
import pickle
from time import sleep
import timeit
import numpy as np

#import snips as snp  # my snippets
#snp.prettyplot(matplotlib)  # my aesthetic preferences for plotting
%matplotlib inline

In [2]:
def req(msg, slp=0.2):
    """Make fault tolerant BGG server requests."""
    # Sleep to make sure you are not pinging the server to frequently
    sleep(slp)
    # Keep trying requests until status-code is 200
    status_code = 500
    while status_code != 200:
        sleep(slp)
        try:
            r = requests.get(msg)
            status_code = r.status_code
            # if status_code != 200:
                # print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        except:
            # print("An exception has occurred, probably a momentory loss of connection. Waiting three seconds...")
            sleep(3)
    return r

In [3]:
# sometimes your connection will blink out in the middle of reading a response, so it's good to 
# wrap requests.get() in something a little more fault tolerant:

def request(msg, slp=1):
    '''A wrapper to make robust https requests.'''
    status_code = 500  # Want to get a status-code of 200
    while status_code != 200:
        sleep(slp)  # Don't ping the server too often
        try:
            r = requests.get(msg)
            status_code = r.status_code
            if status_code != 200:
                print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        except:
            print("An exception has occurred, probably a momentory loss of connection. Waiting one seconds...")
            sleep(1)
    return r

# Scrape dataframe to hold list of boardgames 
Only games with >1000 ratings

In [65]:
# I can specify that I want to browse a list sorted by number of ratings, in descending order, 
# and in this way I can cut off the scraping when I reach whatever ratings sparsity I want 
# (maybe somewhere around 20 ratings per game?)
# For now the only info I'll pull is the game ID (an integer), the game name, the number
# of ratings, and the URL for a thumbnail pic of the game.

# Initialize a DF to hold all our scraped game info
df_all = pd.DataFrame(columns=["id", "name", "nrate", "pic_url"])
min_nrate = 1e5
npage = 1

# Scrap successful pages in the results until we get down to games with < 1000 ratings each
while min_nrate > 1000:
    # Get full HTML for a specific page in the full listing of boardgames sorted by 
    r = request("https://boardgamegeek.com/browse/boardgame/page/%i?sort=numvoters&sortdir=desc" % (npage,))
    soup = BeautifulSoup(r.text, "html.parser")    
    
    # Get rows for the table listing all the games on this page
    table = soup.find_all("tr", attrs={"id": "row_"})  # Get list of all the rows (tags) in the list of games on this page
    df = pd.DataFrame(columns=["id", "name", "nrate", "pic_url"], index=range(len(table)))  # DF to hold this pages results
    
    # Loop through each row and pull out the info for that game
    for idx, row in enumerate(table):
        # Row may or may not start with a "boardgame rank" link, if YES then strip it
        links = row.find_all("a")
        if "name" in links[0].attrs.keys():
            del links[0]
        gamelink = links[1]  # Get the relative URL for the specific game
        gameid = int(gamelink["href"].split("/")[2])  # Get the game ID by parsing the relative URL
        gamename = gamelink.contents[0]  # Get the actual name of the game as the link contents
        imlink = links[0]  # Get the URL for the game thumbnail
        thumbnail = imlink.contents[0]["src"]

        ratings_str = row.find_all("td", attrs={"class": "collection_bggrating"})[2].contents[0]
        nratings = int("".join(ratings_str.split()))

        df.iloc[idx, :] = [gameid, gamename, nratings, thumbnail]

    # Concatenate the results of this page to the master dataframe
    min_nrate = df["nrate"].min()  # The smallest number of ratings of any game on the page
    print("Page %i scraped, minimum number of ratings was %i" % (npage, min_nrate))
    df_all = pd.concat([df_all, df], axis=0)
    npage += 1
    sleep(2) # Keep the BGG server happy.

Page 1 scraped, minimum number of ratings was 16337
Page 2 scraped, minimum number of ratings was 10976
Page 3 scraped, minimum number of ratings was 7602
Page 4 scraped, minimum number of ratings was 6070
Page 5 scraped, minimum number of ratings was 4854
Page 6 scraped, minimum number of ratings was 4188
Page 7 scraped, minimum number of ratings was 3579
Page 8 scraped, minimum number of ratings was 3137
Page 9 scraped, minimum number of ratings was 2769
Page 10 scraped, minimum number of ratings was 2484
Page 11 scraped, minimum number of ratings was 2264
Page 12 scraped, minimum number of ratings was 2052
Page 13 scraped, minimum number of ratings was 1849
Page 14 scraped, minimum number of ratings was 1711
Page 15 scraped, minimum number of ratings was 1586
Page 16 scraped, minimum number of ratings was 1483
Page 17 scraped, minimum number of ratings was 1398
Page 18 scraped, minimum number of ratings was 1316
Page 19 scraped, minimum number of ratings was 1238
Page 20 scraped, mi

In [69]:
df = df_all.copy()
# Reset the index since we concatenated a bunch of DFs with the same index into one DF
df.reset_index(inplace=True, drop=True)
# Write the DF to .csv for future use
df.to_csv("bgg_gamelist.csv", index=False, encoding="utf-8")
print(df.shape)
df.head()

(2300, 4)


,id,name,nrate,pic_url
0,13,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
1,822,Carcassonne,77239,https://cf.geekdo-images.com/micro/img/z0tTaij...
2,30549,Pandemic,74434,https://cf.geekdo-images.com/micro/img/0m3-oqB...
3,36218,Dominion,63194,https://cf.geekdo-images.com/micro/img/VYp2s2f...
4,68448,7 Wonders,61613,https://cf.geekdo-images.com/micro/img/h-Ejv31...


In [67]:
print("Number of games with > 1000 ratings is approximately %i" % (len(df),))
print("Total number of ratings from all these games is %i" % (df["nrate"].sum(),))

Number of games with > 1000 ratings is approximately 2300
Total number of ratings from all these games is 9964941


In [70]:
# Getting the ratings of specific board game (each is in a "comment" div) with API:
r = request("http://www.boardgamegeek.com/xmlapi2/thing?id=13&ratingcomments=1&page=1")
soup = BeautifulSoup(r.text, "xml")
comments = soup("comment")
comments

[<comment rating="10" username="trojasmic" value=""/>,
 <comment rating="10" username="boardsandbourbon" value=""/>,
 <comment rating="10" username="Davazquezg" value=""/>,
 <comment rating="10" username="aloharover" value=""/>,
 <comment rating="10" username="BrikkSteele" value=""/>,
 <comment rating="10" username="tlaraver" value=""/>,
 <comment rating="10" username="itssick" value=""/>,
 <comment rating="10" username="sinahero" value="The best game in the world"/>,
 <comment rating="10" username="Bdhorning" value=""/>,
 <comment rating="10" username="benjamin180" value=""/>,
 <comment rating="10" username="theodoreh" value=""/>,
 <comment rating="10" username="AleBanjo" value=""/>,
 <comment rating="10" username="maxperformance" value=""/>,
 <comment rating="10" username="SunburstRomeo" value=""/>,
 <comment rating="10" username="Bizcho" value=""/>,
 <comment rating="10" username="mirzonja03" value=""/>,
 <comment rating="10" username="Cayden101" value="I've played this game probabl

# Scrape "toy" data set to test scraping loop
First make sure the scraping loop works before running it to scrape all of the board game data.

In [4]:
#Read in csv file created previously
df = pd.read_csv('bgg_gamelist.csv')
df.head()

,id,name,nrate,pic_url
0,13,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
1,822,Carcassonne,77239,https://cf.geekdo-images.com/micro/img/z0tTaij...
2,30549,Pandemic,74434,https://cf.geekdo-images.com/micro/img/0m3-oqB...
3,36218,Dominion,63194,https://cf.geekdo-images.com/micro/img/VYp2s2f...
4,68448,7 Wonders,61613,https://cf.geekdo-images.com/micro/img/h-Ejv31...


In [81]:
# Restrict the data set to fewer games that have fewer ratings (our toy data set!)
#df_toy = df.head(3)
#.loc[df["nrate"] < 2000, ].copy()
#df_toy

,id,name,nrate,pic_url
0,13,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
1,822,Carcassonne,77239,https://cf.geekdo-images.com/micro/img/z0tTaij...
2,30549,Pandemic,74434,https://cf.geekdo-images.com/micro/img/0m3-oqB...


In [7]:
# Restrict the data set to fewer games that have fewer ratings (our toy data set!)
df_toy = df.loc[df["nrate"] < 2000, ].copy()


In [8]:
import numpy as np
# Prepare a "# of FULL pages of ratings" column to track # API calls needed
df_toy["nfullpages"] = (df_toy["nrate"]-50).apply(round, ndigits=-2)/100  # Round DOWN to nearest 100
df_toy.head()

,id,name,nrate,pic_url,nfullpages
1215,218127,Terraforming Mars: Hellas & Elysium,1999,https://cf.geekdo-images.com/micro/img/P7TDzcf...,19.0
1216,171011,Favor of the Pharaoh,1997,https://cf.geekdo-images.com/micro/img/KeMIPQA...,19.0
1217,29937,König von Siam,1994,https://cf.geekdo-images.com/micro/img/CFqARxY...,19.0
1218,37400,Sushizock im Gockelwok,1993,https://cf.geekdo-images.com/micro/img/lH6YKVe...,19.0
1219,176,Give Me the Brain!,1990,https://cf.geekdo-images.com/micro/img/B-Xr9EC...,19.0


In [36]:
### Writing the results to SQL database
# Create the database and make a cursor to talk to it.
import sqlite3
connex = sqlite3.connect("bgg_ratings_comments_toy.db")  # Opens file if exists, else creates file
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results

In [37]:
#############################################################
# Gathering all ratings from all games in toy data set
#############################################################
# Get ratings page-by-page for all games, but do it in chunks of 250 games
for nm, grp in df_toy.groupby(np.arange(len(df_toy))//150):
    # Initialize a DF to hold all the responses for this chunk of games
    df_ratings = pd.DataFrame(columns=["gameid", "username", "rating", "value"], index=range(grp["nrate"].sum()+100000))
        # ***Added "value" column for the actual comments
    
    # Initialize indices for writing to the ratings dataframe
    dfidx_start = 0
    dfidx = 0
    
    # For this group of games, make calls until all FULL pages of every game have been pulled
    pagenum = 1
    while len(grp[grp["nfullpages"] > 0]) > 0: 
        # Get a restricted DF with only still-active games (have ratings pages left)
        active_games = grp[grp["nfullpages"] > 0]

        # Set the next chunk of the DF "gameid" column using the list of game IDs
        id_list = []
        for game in active_games["id"]:
            id_list += [game]*100
        dfidx_end = dfidx_start + len(active_games)*100
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("gameid")] = id_list

        # Make the request with the list of all game IDs that have ratings left
        id_strs = [str(gid) for gid in active_games["id"]]
        gameids = ",".join(id_strs)
        sleep(1.5)  # Keep the server happy
        r = request("http://www.boardgamegeek.com/xmlapi2/thing?id=%s&ratingcomments=1&page=%i" % (gameids, pagenum))
#         while r.status_code != 200:
#             sleep(2)  # Keep the server happy
#             print("Server Error! Response Code %i. Retrying..." % (r.status_code))
#             r = requests.get("http://www.boardgamegeek.com/xmlapi2/thing?id=%s&ratingcomments=1&page=%i" % (gameids, pagenum))        
        soup = BeautifulSoup(r.text, "xml")
        comments = soup("comment")
#         print("Response status was %i - number of ratings retrieved was %i" % (r.status_code, len(comments)))

        # Parse the response and assign it into the dataframe
        l1 = [0]*len(active_games)*100
        l2 = [0]*len(active_games)*100
        l3 = [0]*len(active_games)*100 #For comment (i.e., "value")
        j = 0
        for comm in comments:
            l1[j] = comm["username"]
            l2[j] = float(comm["rating"])
            l3[j] = str(comm["value"]) # ***Added this (to encode comment)
            j += 1
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("username")] = l1
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("rating")] = l2
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("value")] = l3 # ***Added this

        
        grp["nfullpages"] -= 1  # Decrement the number of FULL pages of each game id
        dfidx_start = dfidx_end     
        pagenum += 1  
        print("pagenum updated to %i" %(pagenum,))
    
    # Strip off the empty rows
    df_ratings = df_ratings.dropna(how="all")
    # Write this batch of all FULL pages of ratings for this chunk of games to the DB
    df_ratings.to_sql(name="data", con=connex, if_exists="append", index=False)    
    print("Processed ratings for batch #%i of games." % (nm))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


pagenum updated to 2
pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
pagenum updated to 8
pagenum updated to 9
pagenum updated to 10
pagenum updated to 11
pagenum updated to 12
pagenum updated to 13
pagenum updated to 14
pagenum updated to 15
pagenum updated to 16
pagenum updated to 17
pagenum updated to 18
pagenum updated to 19
pagenum updated to 20
Processed ratings for batch #0 of games.
pagenum updated to 2
pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
pagenum updated to 8
pagenum updated to 9
pagenum updated to 10
pagenum updated to 11
pagenum updated to 12
pagenum updated to 13
pagenum updated to 14
pagenum updated to 15
pagenum updated to 16
pagenum updated to 17
pagenum updated to 18
Processed ratings for batch #1 of games.
pagenum updated to 2
pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
pagenum updated 

In [38]:
### See if it worked. 
connex = sqlite3.connect("bgg_ratings_comments_toy.db")  # Opens file if exists, else creates file
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results
sql = "SELECT * FROM data" + ";"
df_ratings = pd.read_sql_query(sql, connex)
print(df_ratings.shape)
df_ratings.sample(n=10)

(1451100, 4)


,gameid,username,rating,value
181754,32341,Clerebout,6.0,
1055194,1768,Noah Kolman,6.0,
122418,939,panzer-attack,8.0,By far the best Star Wars game I've played and...
9513,228341,weirdojax,10.0,
998695,181817,adrian142,9.0,
619289,111,Lucawahid,7.0,
1014278,141,kkuri,7.0,
287385,154509,Fatesshadow,8.5,
203338,156091,aruom,7.0,
818634,153097,Kapi,7.0,


#### See if Testing Worked

In [34]:
### See if testing worked. 
# Opens file if exists, else creates file
connex = sqlite3.connect("bgg_ratings_comments_toy_test.db")  
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results
sql = "SELECT * FROM data" + ";"
print(df_ratings.shape)
df_ratings = pd.read_sql_query(sql, connex)
df_ratings.sample(n=10)

(108500, 4)


,gameid,username,rating,value
79171,67051,talismanisland,10.00,
51182,198454,Trojda,9.00,
79424,10496,msweazey,9.00,Very entertaining and thematic Arthurian legen...
15387,38531,tasajara,8.50,
70096,300,raythornton,7.50,
5142,197831,Bandit650S,10.00,
50454,175156,horfrost3,10.00,Jg xmas 16 (tg)
78028,110831,Alkonyi,10.00,
52017,130882,maciejgoldfarth,9.00,
68978,192934,rahdo,8.34,only got rid of it because of international mo...


In [39]:
df_ratings.head()

,gameid,username,rating,value
0,218127,nkhvorov,10.0,
1,218127,barefootbear,10.0,
2,218127,jliston582,10.0,
3,218127,DungeonCrawl,10.0,
4,218127,Sparticus09,10.0,


In [43]:
#### get rid of empty comments and then see what we're working with:
df_comments = df_ratings[df_ratings.value != '']
print(df_comments.shape)
df_comments.head()

(311383, 4)


,gameid,username,rating,value
21,218127,Rubeus,10.0,First expansion that we buy for this extraordi...
67,218127,JMrawrks,10.0,I love maps.in.space
104,171011,jonky,10.0,This game is great. I think people are cuckoo ...
110,171011,survtek,10.0,An awesome dice building game! This is my per...
115,171011,AquaSulis,10.0,Nov 2015 - 10 ... brilliant dice chucking game...


In [35]:
# Save our changes and close the connection
connex.commit()
connex.close()

In [19]:
import sqlite3 
# Opens file if exists, else creates file
connex = sqlite3.connect("bgg_ratings_comments_toy_test.db")  
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results
sql = "SELECT * FROM data" + ";"

In [20]:
# DataFrame it: 
df_ratings = pd.read_sql_query(sql, connex)
df_ratings.sample(n=10)

,gameid,username,rating
169264,66646,Ginja Ninja,7.0
720286,38548,zeeb13,5.0
398564,9341,hanuman,7.0
88001,40769,aboveriver,7.5
1282200,112869,necronikos,10.0
28116,5406,Gregarius,8.0
137859,108044,Bulsara,6.8
1289180,197586,jwml,9.5
853578,3522,tromafiend02,2.0
2005,6795,Papyrus,10.0


# Get all board game data (not testing anymore)
(All board games included in list on bgg_gamelist.csv)

In [4]:
#Read in csv file created previously
df = pd.read_csv('bgg_gamelist.csv')
print(df.shape)
df.head()

(2300, 4)


,id,name,nrate,pic_url
0,13,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
1,822,Carcassonne,77239,https://cf.geekdo-images.com/micro/img/z0tTaij...
2,30549,Pandemic,74434,https://cf.geekdo-images.com/micro/img/0m3-oqB...
3,36218,Dominion,63194,https://cf.geekdo-images.com/micro/img/VYp2s2f...
4,68448,7 Wonders,61613,https://cf.geekdo-images.com/micro/img/h-Ejv31...


In [5]:
import numpy as np
# Prepare a "# of FULL pages of ratings" column to track # API calls needed
df["nfullpages"] = (df["nrate"]-50).apply(round, ndigits=-2)/100  # Round DOWN to nearest 100
df.head()

,id,name,nrate,pic_url,nfullpages
0,13,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,775.0
1,822,Carcassonne,77239,https://cf.geekdo-images.com/micro/img/z0tTaij...,772.0
2,30549,Pandemic,74434,https://cf.geekdo-images.com/micro/img/0m3-oqB...,744.0
3,36218,Dominion,63194,https://cf.geekdo-images.com/micro/img/VYp2s2f...,631.0
4,68448,7 Wonders,61613,https://cf.geekdo-images.com/micro/img/h-Ejv31...,616.0


In [6]:
### Writing the results to SQL database
# Create the database and make a cursor to talk to it.
import sqlite3
connex = sqlite3.connect("bgg_ratings_comments_full_r1.db")  # Opens file if exists, else creates file
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results

In [7]:
#############################################################
# Gathering all ratings from all games (with ~1000 ratings)
#############################################################
# Get ratings page-by-page for all games, but do it in chunks of 250 games
for nm, grp in df.groupby(np.arange(len(df))//50):
    # Initialize a DF to hold all the responses for this chunk of games
    df_ratings = pd.DataFrame(columns=["gameid", "username", "rating", "value"], index=range(grp["nrate"].sum()+1000000))
        # ***Added "value" column for the actual comments
    
    # Initialize indices for writing to the ratings dataframe
    dfidx_start = 0
    dfidx = 0
    
    # For this group of games, make calls until all FULL pages of every game have been pulled
    pagenum = 1
    while len(grp[grp["nfullpages"] > 0]) > 0: 
        # Get a restricted DF with only still-active games (have ratings pages left)
        active_games = grp[grp["nfullpages"] > 0]

        # Set the next chunk of the DF "gameid" column using the list of game IDs
        id_list = []
        for game in active_games["id"]:
            id_list += [game]*100
        dfidx_end = dfidx_start + len(active_games)*100
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("gameid")] = id_list

        # Make the request with the list of all game IDs that have ratings left
        id_strs = [str(gid) for gid in active_games["id"]]
        gameids = ",".join(id_strs)
        sleep(2)  # Keep the server happy
        r = request("http://www.boardgamegeek.com/xmlapi2/thing?id=%s&ratingcomments=1&page=%i" % (gameids, pagenum))
#         while r.status_code != 200:
#             sleep(2)  # Keep the server happy
#             print("Server Error! Response Code %i. Retrying..." % (r.status_code))
#             r = requests.get("http://www.boardgamegeek.com/xmlapi2/thing?id=%s&ratingcomments=1&page=%i" % (gameids, pagenum))        
        soup = BeautifulSoup(r.text, "xml")
        comments = soup("comment")
#         print("Response status was %i - number of ratings retrieved was %i" % (r.status_code, len(comments)))

        # Parse the response and assign it into the dataframe
        l1 = [0]*len(active_games)*100
        l2 = [0]*len(active_games)*100
        l3 = [0]*len(active_games)*100 #For comment (i.e., "value")
        j = 0
        for comm in comments:
            l1[j] = comm["username"]
            l2[j] = float(comm["rating"])
            l3[j] = str(comm["value"]) # ***Added this (to encode comment)
            j += 1
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("username")] = l1
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("rating")] = l2
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("value")] = l3 # ***Added this

        
        grp["nfullpages"] -= 1  # Decrement the number of FULL pages of each game id
        dfidx_start = dfidx_end     
        pagenum += 1  
        print("pagenum updated to %i" %(pagenum,))
    
    # Strip off the empty rows
    df_ratings = df_ratings.dropna(how="all")
    # Write this batch of all FULL pages of ratings for this chunk of games to the DB
    df_ratings.to_sql(name="data", con=connex, if_exists="append", index=False)    
    print("Processed ratings for batch #%i of games." % (nm))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


pagenum updated to 2
pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 504. Retrying...
pagenum updated to 7
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Respo

Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 504. Retrying...
pagenum updated to 19
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 504. Retrying...
pagenum updated to 20
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Erro

Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 504. Retrying...
pagenum updated to 34
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 504. 

Server Error! Response Code 500. Retrying...
Server Error! Response Code 504. Retrying...
pagenum updated to 48
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 504. Retrying...
pagenum updated to 49
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Error! Response Code 500. Retrying...
Server Erro

...

pagenum updated to 18
Server Error! Response Code 504. Retrying...
pagenum updated to 19
Server Error! Response Code 504. Retrying...
pagenum updated to 20
Server Error! Response Code 504. Retrying...
pagenum updated to 21
Server Error! Response Code 504. Retrying...
pagenum updated to 22
Server Error! Response Code 504. Retrying...
pagenum updated to 23
Server Error! Response Code 504. Retrying...
pagenum updated to 24
pagenum updated to 25
Server Error! Response Code 504. Retrying...
pagenum updated to 26
Server Error! Response Code 504. Retrying...
pagenum updated to 27
pagenum updated to 28
Server Error! Response Code 504. Retrying...
pagenum updated to 29
Server Error! Response Code 504. Retrying...
pagenum updated to 30
Server Error! Response Code 504. Retrying...
pagenum updated to 31
Server Error! Response Code 504. Retrying...
pagenum updated to 32
Server Error! Response Code 504. Retrying...
pagenum updated to 33
Server Error! Response Code 504. Retrying...
pagenum updated to

pagenum updated to 33
pagenum updated to 34
pagenum updated to 35
pagenum updated to 36
pagenum updated to 37
pagenum updated to 38
pagenum updated to 39
pagenum updated to 40
pagenum updated to 41
pagenum updated to 42
pagenum updated to 43
pagenum updated to 44
pagenum updated to 45
pagenum updated to 46
pagenum updated to 47
pagenum updated to 48
pagenum updated to 49
pagenum updated to 50
pagenum updated to 51
pagenum updated to 52
pagenum updated to 53
pagenum updated to 54
pagenum updated to 55
pagenum updated to 56
pagenum updated to 57
pagenum updated to 58
pagenum updated to 59
pagenum updated to 60
pagenum updated to 61
pagenum updated to 62
pagenum updated to 63
pagenum updated to 64
pagenum updated to 65
pagenum updated to 66
pagenum updated to 67
pagenum updated to 68
pagenum updated to 69
pagenum updated to 70
pagenum updated to 71
pagenum updated to 72
pagenum updated to 73
pagenum updated to 74
pagenum updated to 75
pagenum updated to 76
Processed ratings for batch #6 o

pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
pagenum updated to 8
pagenum updated to 9
pagenum updated to 10
pagenum updated to 11
pagenum updated to 12
pagenum updated to 13
pagenum updated to 14
pagenum updated to 15
pagenum updated to 16
pagenum updated to 17
pagenum updated to 18
pagenum updated to 19
pagenum updated to 20
pagenum updated to 21
pagenum updated to 22
pagenum updated to 23
pagenum updated to 24
pagenum updated to 25
pagenum updated to 26
pagenum updated to 27
pagenum updated to 28
pagenum updated to 29
Server Error! Response Code 504. Retrying...
pagenum updated to 30
pagenum updated to 31
pagenum updated to 32
pagenum updated to 33
pagenum updated to 34
pagenum updated to 35
pagenum updated to 36
pagenum updated to 37
pagenum updated to 38
pagenum updated to 39
pagenum updated to 40
pagenum updated to 41
pagenum updated to 42
Processed ratings for batch #12 of games.
Server Error! Response Code 504. Retryin

pagenum updated to 12
pagenum updated to 13
pagenum updated to 14
pagenum updated to 15
pagenum updated to 16
pagenum updated to 17
pagenum updated to 18
pagenum updated to 19
pagenum updated to 20
pagenum updated to 21
pagenum updated to 22
pagenum updated to 23
pagenum updated to 24
Processed ratings for batch #21 of games.
pagenum updated to 2
pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
An exception has occurred, probably a momentory loss of connection. Waiting one seconds...


pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
pagenum updated to 8
pagenum updated to 9
pagenum updated to 10
pagenum updated to 11
pagenum updated to 12
pagenum updated to 13
pagenum updated to 14
pagenum updated to 15
pagenum updated to 16
Processed ratings for batch #31 of games.
pagenum updated to 2
pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
pagenum updated to 8
pagenum updated to 9
pagenum updated to 10
pagenum updated to 11
pagenum updated to 12
pagenum updated to 13
pagenum updated to 14
pagenum updated to 15
Processed ratings for batch #32 of games.
pagenum updated to 2
pagenum updated to 3
pagenum updated to 4
pagenum updated to 5
pagenum updated to 6
pagenum updated to 7
pagenum updated to 8
pagenum updated to 9
pagenum updated to 10
pagenum updated to 11
pagenum updated to 12
pagenum updated to 13
pagenum updated to 14
pagenum updated to 15
Processed ratings for batch #33 of games.
pagenum updated

In [11]:
# Save our changes and close the connection
connex.commit()
connex.close()

In [9]:
# Next step: Read in final page for all the board games (get that script to work...)
# Open the database again and make a cursor to talk to it.
import sqlite3
connex = sqlite3.connect("bgg_ratings_comments_full_r1.db")  # Opens file if exists, else creates file
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results

In [10]:
#############################################################
# Request the final partial page of ratings for each game
#############################################################
# Restore the correct number of FULL pages
df["nfullpages"] = (df["nrate"]-50).apply(round, ndigits=-2)/100  # Round DOWN to nearest 100, then divide by 100

# Initialize a DF to hold all the responses over all the chunks of games
df_ratings_r2 = pd.DataFrame(columns=["gameid", "username", "rating", "value"], index=range(len(df)*100))
#df_ratings = pd.DataFrame(columns=["gameid", "username", "rating"], index=range(len(df_toy)*100))

# Initialize indices for writing to the ratings dataframe
dfidx_start = 0
dfidx = 0

# Loop through game-by-game and request the final page of ratings for each game
for idx, row in df.iterrows():
    # Get the game ID and the last page number to request
    pagenum = row["nfullpages"] + 1
    gameid = row["id"]
    
    # Make the request for just the last page of ratings of this game
    sleep(2)  # Keep the server happy
    r = requests.get("http://www.boardgamegeek.com/xmlapi2/thing?id=%i&ratingcomments=1&page=%i" % (gameid, pagenum))
    while r.status_code != 200:
        sleep(2)  # Keep the server happy
        print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        r = requests.get("http://www.boardgamegeek.com/xmlapi2/thing?id=%i&ratingcomments=1&page=%i" % (gameid, pagenum))
    soup = BeautifulSoup(r.text, "xml")
    comments = soup("comment")
#         print("Response status was %i - length of comments is %i" % (r.status_code, len(comments)))

    # Set the next chunk of the DF "gameids" column with this gameid
    id_list = [gameid]*len(comments)
    dfidx_end = dfidx_start + len(comments)
    df_ratings_r2.iloc[dfidx_start:dfidx_end, df_ratings_r2.columns.get_loc("gameid")] = id_list

    # Parse the response and assign it into the dataframe
    l1 = [0]*len(comments)
    l2 = [0]*len(comments)
    l3 = [0]*len(comments) # Added
    j = 0
    for comm in comments:
        l1[j] = comm["username"]
        l2[j] = float(comm["rating"])
        l3[j] = str(comm["value"]) # Added
        j += 1
    df_ratings_r2.iloc[dfidx_start:dfidx_end, df_ratings_r2.columns.get_loc("username")] = l1
    df_ratings_r2.iloc[dfidx_start:dfidx_end, df_ratings_r2.columns.get_loc("rating")] = l2
    df_ratings_r2.iloc[dfidx_start:dfidx_end, df_ratings_r2.columns.get_loc("value")] = l3

    dfidx_start = dfidx_end   # Increment the starting index for next round        

    if idx%100 == 0:
        print("Finished with a chunk of 100 games.")
        
# Strip off the empty rows
df_ratings_r2 = df_ratings_r2.dropna(how="all")

# Write this final batch of all partial pages of ratings for this chunk of games to the DB
df_ratings_r2.to_sql(name="data", con=connex, if_exists="append", index=False) 

Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.
Finished with a chunk of 100 games.


In [17]:
### Verify that the hours and hours of scraping worked. 
# Note: Don't close out the sql db connection if 
# you're only going to turn around and reopen it again in a second. 
# Only close if you're not using the connection anymore for a while. 

import sqlite3

# Connecting to the database file
connex = sqlite3.connect("bgg_ratings_comments_full_r1.db")  # Opens file if exists, else creates file
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results

In [26]:
# Select only up to 10 rows that match a certain value in 1 column

cur.execute('SELECT * FROM data WHERE value!="" LIMIT 10')
#ten_rows = cur.fetchall()
#print('Sample of 10 entries:', ten_rows)

# Displays each row on a new line (looks a little nicer)
for row in cur.execute('SELECT * FROM data WHERE value!="" LIMIT 10'):
        print(row)

(13, 'sinahero', '10.0', 'The best game in the world')
(13, 'Cayden101', '10.0', "I've played this game probably close to a hundred times. Favorite game of all time.")
(13, 'Spaceage Polymer', '10.0', "I didn't want to give Catan a perfect ten, because it's not even my favorite game. But I can't think of any flaw to criticize; the worst thing I can think to say about it is that it's slightly annoying that it's the only game my friends ever want to play. Well done, Klays Teuber! Definitely get the expansion for 5-6 players.")
(13, 'asauve19', '10.0', "My all time favorite game. Even if you are rolling bad, have bad tiles, or bad numbers, there are always ways to win. Comebacks and last minute twists are always in store. Also, unlike many games, there are things to do, when it's not your turn to keep inactive players, active.")
(13, 'Feelie', '10.0', 'One of (if not) the best board game. Ever changing dynamics, fantastic expansions, lots of fun and challenge.')
(13, 'Canada91pl', '10.0',

In [ ]:
# Save our changes and close the connection
connex.commit()
connex.close()